In [10]:
import random

# 一共有多少种类型的牌
pai_types = {'WAN_1':1 , 'WAN_2':2, 'WAN_3':3, 'WAN_4':4,'WAN_5':5,'WAN_6':6, 'WAN_7':7, 'WAN_8':8, 'WAN_9':9,
                 'DONGFENG':10, 'NANFENG':11, 'XIFENG':12, 'BEIFENG':13, 'BAIBAN':14, 'FACAI':15,  'HONGZHONG':16}

# 生成一副牌
pais = [v for v in  list(pai_types.values())  for i in range (0, 4) ]

# 打乱顺序
random.shuffle(pais)

# 牌分成两部分，一部分是手上的牌，一部分是从对家碰、吃得到的不能再改变
host_dynamic_pais = [pais.pop() for i in range(0, 13)]
oppo_dynamic_pais = [pais.pop() for i in range(13, 26)]

host_static_pais = []
oppo_static_pais = []

# 庄家和玩家出的所有牌
host_output_pais = []
oppo_output_pais = []

# 随机判断谁先摸牌，TRUE： 代表庄家， FALSE：代表对家
oppo_turn = random.random() >= 0.5
if oppo_turn:
    mopai(oppo_dynamic_pais)
    p = chu_pai(oppo_dynamic_pais)
    oppo_dynamic_pais.remove(p)
    oppo_output_pais.append(p)
else:
    mopai(host_dynamic_pais)
    p = chu_pai(host_dynamic_pais)
    host_dynamic_pais.remove(p)
    host_output_pais.append(p)
oppo_turn = not oppo_turn

# 表示游戏还没有完成
not_finished = True

# 依次进行出牌，直到乙方赢了或没有牌了
while not_finished and len(pais) != 0:
    if(oppo_turn):
        """
        对家出牌先判断能能否碰、吃、杠、胡，如果都不满足则进行摸牌，后再进行出牌
        """

        # 最后一次出牌
        last_pai = host_output_pais[-1]
        
        if is_gang(oppo_dynamic_pais, last_pai):# 杠
            gang(oppo_dynamic_pais, oppo_static_pais, last_pai)
        elif is_peng(oppo_dynamic_pais, last_pai): # 碰
            peng(oppo_dynamic_pais, oppo_static_pais, last_pai)
        elif is_chi(oppo_dynamic_pais, last_pai): # 吃
            chi(oppo_dynamic_pais, oppo_static_pais, last_pai)
        else: # 摸牌
            mopai(oppo_dynamic_pais)

        # 判断是否能胡牌
        if is_hu(oppo_dynamic_pais + oppo_static_pais):
            not_finished = False
            print("对手胡了: ", oppo_dynamic_pais + oppo_static_pais)
        else:# 出牌
            p = chu_pai(oppo_dynamic_pais)
            oppo_dynamic_pais.remove(p)
            oppo_output_pais.append(p)
        
        oppo_turn = False
    else:
        # 最后一次出牌
        last_pai = oppo_output_pais[-1]
        
        if is_gang(host_dynamic_pais, last_pai):# 杠
            gang(host_dynamic_pais, host_static_pais, last_pai)
        elif is_peng(host_dynamic_pais, last_pai): # 碰
            peng(host_dynamic_pais, host_static_pais, last_pai)
        elif is_chi(host_dynamic_pais, last_pai): # 吃
            chi(host_dynamic_pais, host_static_pais, last_pai)
        else: # 摸牌
            mopai(host_dynamic_pais)

        # 判断是否能胡牌
        if is_hu(host_dynamic_pais + host_static_pais):
            not_finished = False
            print("庄胡了: ", host_dynamic_pais + host_static_pais)
        else:# 出牌
            p = chu_pai(host_dynamic_pais)
            host_dynamic_pais.remove(p)
            host_output_pais.append(p)
        
        oppo_turn = True

对手胡了:  [14, 14, 14, 12, 12, 9, 9, 9, 1, 2, 3, 3, 4, 5]


In [9]:
import collections

def mopai(my_pais):
    """
    摸牌，判断牌是否已经摸完
    """
    if len(pais) > 0:
        my_pais.append(pais.pop()) 
    else:
        print("流局")

def is_peng(my_dynamic_pais, pai):
    """
    判断是否可以碰对家出的牌
    """
    return my_dynamic_pais.count(pai) == 2

def peng(my_dynamic_pais, my_static_pais, pai):
    """
    碰牌且把相关牌从动态牌面中移除，添加到静态牌面
    """
   
    removeEle(my_dynamic_pais, pai)
    my_static_pais.extend([pai, pai, pai])

def is_chi(my_pais, pai):
    """
    判断是否可以吃对家出的牌
    """
    if pai <= 9 and pai - 2 in my_pais and pai - 1 in my_pais:
        return True
    elif pai <= 8 and pai - 1 in my_pais and pai + 1 in my_pais:
        return True
    elif pai <= 7 and pai + 1 in my_pais and pai + 2 in my_pais:
        return True
    return False

def chi(my_dynamic_pais, my_static_pais, pai):
    """
    吃牌且把相关牌从动态牌面中移除，添加到静态牌面
    """
    
    if pai <= 9 and  pai - 2 in my_dynamic_pais and pai - 1 in my_dynamic_pais:
        my_static_pais.extend([pai - 2, pai - 1, pai])
        my_dynamic_pais.remove(pai - 1)
        my_dynamic_pais.remove(pai - 2)
    elif pai<= 8 and pai - 1 in my_dynamic_pais and pai + 1 in my_dynamic_pais:
        my_static_pais.extend([pai - 1, pai, pai + 1])
        my_dynamic_pais.remove(pai - 1)
        my_dynamic_pais.remove(pai + 1)
    elif pai <=7 and pai + 1 in my_dynamic_pais and pai + 2 in my_dynamic_pais: 
        my_static_pais.extend([pai, pai + 1, pai + 2])
        my_dynamic_pais.remove(pai + 1)
        my_dynamic_pais.remove(pai + 2)

def removeEle(pais, pai):
    """
    删除指定元素
    """
    for index, p in enumerate(pais):
        if p == pai:
            del pais[index]
            
def is_hu(my_pais):
    """
    TODO 参数只能是手上的牌，碰、吃、明杠的牌不算
    且返回所有的能胡的牌的排列组合
    
    判断是否可以胡牌，牌可以是自己摸的牌或胡对家的牌
    计算每一种胡牌的可能的排列组合
    """
    
    # 每个牌出现的次数
    pai_count = {pai:0 for pai in pai_types.values()}
    for pai in my_pais:
        pai_count[pai] += 1
    
    # 所有可能的对子,计算每种胡的可能
    for i in pai_count.keys():
        if pai_count[i] >= 2:
            pai_count[i] -= 2
            # 这里可以加上番数
            if check_sequence_triplet(pai_count):
                return True 
            pai_count[i] += 2
    return False

def check_sequence_triplet(pai):
    """
    TODO
    需要先确定静态牌，去掉静态牌后的牌能组成的牌，全是对子的还没有包括
    """ 
    
    # 判断是否所有元素已经使用，是：表明已经可以胡了
    success = True
    for i in pai.values():
        if i != 0:
            success = False
            break
    if success:
        return True
    
    # i 表示牌的编号
    for i in pai.keys():
        # 判断有没有顺子
        sunzi = i + 2 <= 9 and pai[i] >= 1 and pai[i + 1] >= 1 and pai[i + 2] >= 1
        if (sunzi):
            pai[i] -= 1
            pai[i + 1] -= 1
            pai[i + 2] -=1
            if check_sequence_triplet(pai):
                return True
            pai[i] += 1
            pai[i + 1] += 1
            pai[i + 2] += 1
        
        # 判断有没有刻子
        if pai[i] >= 3:
            pai[i] -= 3
            if check_sequence_triplet(pai):
                return True
            pai[i] += 3
    return False

def is_gang(my_pais, pai):
    """
    判断是否可以杠
    """
    return my_pais.count(pai) == 3

def gang(my_dynamic_pais, my_static_pais, pai):
    """
    杠
    """
    removeEle(my_dynamic_pais, pai)
    my_static_pais.extend([pai, pai, pai, pai])
    mopai(my_dynamic_pais)

def chu_pai(my_pais):
    """
    出牌逻辑，这部分待优化，出单牌
    """
    pai_count = collections.Counter(my_pais)
    for (k,v) in pai_count.items():
        if v == 1:
            return k
    return my_pais[0]

def cal_score(pais):
    """
    计算牌的得分
    """
    
    pass

In [150]:
import numpy as np


def isKezi():

{
    gang:{
      1: 0, # 0 表示杠的别人的牌，1 表示暗杠
    },
    kezi:{
        11: 0,
        2: 1
    },
    sunzi:{
         7: 0, # 7 表示 7， 8， 9
         4: 1,
    },
    jiangpai:[3, 4]
}

def isDasixi(pais):
    """
    大四喜，4 副风刻（杠）加一对将牌组成的牌型
    """
    pais.gang.get(10, 0) + pais.gang.get(11, 0) + pais.gang.get(12, 0)
    return pais.count(pai_types['DONGFENG']) > 2 \
        and pais.count(pai_types['XIFENG']) > 2 \
        and pais.count(pai_types['NANFENG']) > 2 \
        and pais.count(pai_types['BEIFENG']) > 2

def isDasanyuan(pais):
    """
    大三元，牌里有中、发、白 3 副刻子
    """
    return pais.count(pai_types['HONGZHONG']) > 2 and pais.count(pai_types['FACAI']) > 2 and pais.count(pai_types['BAIBAN']) > 2

def isJiubaoliandeng(pais, pai):
    """
    pais ：自己手上的牌， pai ：对手出的牌
    九宝莲灯，由一种花色序数牌子按1112345678999组成的特定牌型，见同花色任何1张序数牌即成胡牌
    """
    pais.sort() # 按照由小到大排序
    return pais == [1, 1, 1, 2, 3, 4, 5, 6, 7, 8, 9, 9, 9] and pai <= 9

def isShibaluoghan(pais):
    """
    胡牌时，牌里有 4 副杠，明暗杠均可
    """
    pai_count = collections.Counter(pais)
    return len(list(filter(lambda x: x == 4, pai_count.values()))) == 4

def isQixinlianzhu(pais):
    """
    七星连珠，由一种花色序数牌且序数相连的 7 个对子组成的牌型
    """
    pais.sort()
    return pais in [[1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6, 7, 7],\
                    [2, 2, 3, 3, 4, 4, 5, 5, 6, 6, 7, 7, 8, 8],\
                    [3, 3, 4, 4, 5, 5, 6, 6, 7, 7, 8, 8, 9, 9]]

def isBeidouqixin(pais):
    """
    北斗七星，胡牌时，由 7 对字牌组成
    """
    
    pai_count = collections.Counter(pais)
    for k, v in pai_count.items():
        # 1. 判断是否全是字牌，且判断每个字牌都是两个
        if k < 9 or v != 2:
            return False
    return True

def isXiaosixi(pais):
    """
    小四喜，牌里有风牌的 3 副刻子及将牌
    """
    pai_count = collections.Counter(pais)
    return len(list(filter(lambda x: x[0] >= 10 and x[0] <= 13 and x[1] == 3, pai_count.items()))) == 3\
        and len(list(filter(lambda x: x[0] >= 10 and x[0] <= 13 and x[1] == 2, pai_count.items()))) == 1
    
def isXiaosanyuan(pais):
    """
    小三元，牌里有箭牌的两副刻子及将牌
    """
    pai_count = collections.Counter(pais)
    return len(list(filter(lambda x: x[0] >= 14 and x[1] == 3, pai_count.items()))) == 2\
        and len(list(filter(lambda x: x[0] >= 14 and x[1] == 2, pai_count.items()))) == 1    
    
def isGuoxuedashi(pais):
    """
    国学大师，牌型由字牌的刻子（杠）、将组成
    """
    pai_count = collections.Counter(pais)
    return len(list(filter(lambda x: x[0] >= 10 and x[1] >= 3, pai_count.items()))) == 4\
        and len(list(filter(lambda x: x[0] >= 10 and x[1] == 2, pai_count.items()))) == 1  
    
def isSianke(pais):
    """
    四暗刻，牌里有4个暗刻（暗杠）
    TODO 怎么判断是暗刻？
    """
    pai_count = collections.Counter(pais)
    return len(list(filter(lambda x: x[1] >= 3, pai_count.items()))) == 4

"""
天胡、地胡
"""

def isSangang(pais):
    """
    三杠，牌里有3副杠，明暗杠均可
    """
    pai_count = collections.Counter(pais)
    return len(list(filter(lambda x: x[1] == 4, pai_count.items()))) == 3

def isHunyaojiu(pais):
    """
    混幺九，由字牌和序数牌一、九的刻子及将牌组成的牌型
    """
    pai_count = collections.Counter(pais)
    
    # 判断是否在指定的类型内
    types = [1, 9, 10, 11, 12, 13, 14, 15, 16]
    for k in pai_count.keys():
        if k not in types:
            return False
   
    # 判断是否是有刻子和将来组成
    return len(list(filter(lambda x: x[1] == 3, pai_count.items()))) == 4 

def isQidui(pais):
    """
    七对，牌型由 7 个对子组成
    """
    pai_count = collections.Counter(pais)
    return len(list(filter(lambda x: x[1] == 2, pai_count.items()))) == 7

def isQingyise(pais):
    """
    清一色，牌型由一种花色的序数牌组成
    """
    for pai in pais:
        if pai > 9:
            return False
    return True

"""
TODO 三暗刻 清龙
"""
def isQinglong(pais):
    """
    清龙，有 123，456，789 三付顺子即可
    """
    for pai in pais:
        if pai > 9:
            return False
    return True

def isDayu5(pais):
    """
    大于 5 牌型由序数牌 6-9 的顺子、刻子、将牌组成
    """
    for pai in pais:
        if pai <= 5 or pai > 9:
            return False
    return True

def isXiaoyu5(pais):
    """
    小于 5 牌型由序数牌1-4的顺子、刻子、将牌组成
    """
    for pai in pais:
        if pai >= 5:
            return False
    return True

def isSanfengke(pais):
    """
    三风刻, 牌里有3个风刻
    """
    pai_count = collections.Counter(pais)
    return len(list(filter(lambda x: x[0] in [10, 11, 12, 13] and x[1] == 3, pai_count.items()))) == 3

def isSanfengke(pais):
    """
    三风刻, 牌里有3个风刻
    """
    pai_count = collections.Counter(pais)
    return len(list(filter(lambda x: x[0] in [10, 11, 12, 13] and x[1] == 3, pai_count.items()))) == 3

def isHaidilaoyue(pais):
    """
    海底捞月，自摸或胡对方打出的最后一张牌
    """
    pass

def isHaidilaoyue(pais):
    """
    海底捞月，自摸或胡对方打出的最后一张牌
    """
    pass

def isGangshangkaihua(pais):
    """
    杠上开花，开杠抓进的牌成胡牌（不包括补花）
    """
    pass

def isPengpenghu(pais):
    """
    碰碰胡，牌型由 4 副刻子（或杠）、将牌组成
    """
    pai_count = collections.Counter(pais)
    return len(list(filter(lambda x: x[1] >= 3, pai_count.items()))) == 4

def isHunyise(pais):
    """
    混一色，牌型万字牌及字牌组成
    """
    pass

def isQuanqiuren(pais):
    """
    全求人，胡牌时，全靠吃牌、碰牌、单钓别人打出的牌胡牌
    """
    pass

def isQuanangang(pais):
    """
    全暗杠，牌里至少两幅暗杠
    """
    pass

def isShuangjianke(pais):
    """
    双箭刻, 牌里有 2 副箭刻（或杠）
    """
    pai_count = collections.Counter(pais)
    return len(list(filter(lambda x: x[0] >= 14 and  x[1] >= 3, pai_count.items()))) == 2

def isQuandaiyao():
    """
    全带幺，胡牌时，每副牌、将牌都有幺牌。（胡牌时各组牌除了字牌都必须有一或九的序数牌）
    """
    pass

def isBuQiuren(pais):
    """
    不求人，4副牌及将中没有吃牌、碰牌（包括明杠），自摸胡牌。
    """
    pass

def isShuangminggang():
    """
    双明杠，牌里有 2 个明杠
    """
    pass

def isTingpai():
    """
    听牌，玩家听牌则不可换牌，自动摸打
    """
    pass

def isJianke(pais):
    """
    牌里有中、发、白，这3个牌中的任一个牌组成的 1 副刻子
    """
    pai_count = collections.Counter(pais)
    return len(list(filter(lambda x: x[0] >= 14 and  x[1] == 3, pai_count.items()))) >= 1

def isJianke(pais):
    """
    牌里有中、发、白，这3个牌中的任一个牌组成的 1 副刻子
    """
    pai_count = collections.Counter(pais)
    return len(list(filter(lambda x: x[0] >= 14 and  x[1] == 3, pai_count.items()))) >= 1

def isMenqianqing(pais):
    """
    门前清，没有吃、碰、明杠，胡别人打出的牌
    """
    pass

def isSiguiyi():
    """
    四归一，牌里有 4 张相同的牌归于一家的顺、刻子、对、将牌（不包括杠牌）
    """
    pass

def isShuanganke():
    """
    双暗刻，牌里有2个暗刻
    """
    pass

def isAngang():
    """
    暗杠，牌里有一副自抓4张相同的牌且开杠
    """
    pass

def isDuan19(pais):
    """
    断幺九，牌里没有一、九牌及字牌
    """
    for pai in pais:
        if pai in [1, 9, 10, 11, 12, 13, 14, 15, 16]:
            return False
    return True

def isLian6(pais):
    """
    连六，一种花色六张序数相连的顺子
    """
    pass

def isLaoshaofu(pais):
    """
    老少一副，牌里花色相同的123、789的顺子各一副
    """
    pass

def isYaojiuke(pais):
    """
    幺九刻，牌里有序数为一、九的一副刻子（杠）或是字牌的一副刻子（杠）
    """
    pai_count = collections.Counter(pais)
    return len(list(filter(lambda x: x[1] >=3, pai_count.items()))) >= 1


def isMinggang(pais):
    """
    明杠, 自己有暗刻，碰别人打出的一张相同的牌开杠：或自己抓进一张与碰的明刻相同
    """
    pass

def isDanzhang(pais):
    """
    单张，单胡123的3及789的7或1233和3、7789和7都为边张。手中有12345或1112胡3，56789或8889胡7不算边张。
    """
    pass

def isKanzhang(pais):
    """
    坎张，看涨胡牌说明：胡牌时，胡2张牌之间的牌。4556和5也为坎张，手中有45567胡6不算坎张。不计边张、单调将。
    """
    pass

def isDandiaojiang(pais):
    """
    单钓将，看涨胡牌说明：钓单张牌作将成胡。不计边张坎张。
    """
    pass

def isZimo(pais):
    """
    自摸，自己摸的牌胡
    """
    pass

In [152]:
pais = np.array([1, 1, 10, 10, 10, 11, 11,11, 14,14, 14,6,6,6])
# isShibaluoghan(pais)


print(isDuan19(pais))

False


In [153]:
print(None +1)

TypeError: unsupported operand type(s) for +: 'NoneType' and 'int'